# QA + Confluence

Полезные ссылки:
* https://atlassian-python-api.readthedocs.io/confluence.html
* https://developer.atlassian.com/cloud/confluence/rest/v2/api-group-page/#api-pages-id-get
* https://developer.atlassian.com/server/confluence/advanced-searching-using-cql/
* https://spacy.io/usage/spacy-101
* https://python.langchain.com/docs/use_cases/question_answering/

## Установка зависимостей


In [41]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
%pip install -U langchain pypdf langchain-community faiss-cpu tiktoken gigachat beautifulsoup4 pytesseract pdf2image lxml --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install nltk spacy -U --quiet
!python -m spacy download ru_core_news_sm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB 1.3 MB/s eta 0:00:12
     --------------------------------------- 0.1/15.3 MB 762.6 kB/s eta 0:00:20
     ---------------------------------------- 0.1/15.3 MB 1.0 MB/s eta 0:00:15
      --------------------------------------- 0.2/15.3 MB 1.3 MB/s eta 0:00:12
     - -------------------------------------- 0.4/15.3 MB 1.8 MB/s eta 0:00:09
     - -------------------------------------- 0.6/15.3 MB 1.9 MB/s eta 0:00:08
     -- ------------------------------------- 0.9/15.3 MB 2.8 MB/s eta 0:00:06
     -- ------------------------------------- 0.9/15.3 MB 2.8 MB/s eta 0:00:06
     --- ------------------------------------ 1.4/15.3 MB 3.6 MB/s eta 0:00:04
     --- ------------------------------------ 1.5/15.3 MB 3.2 MB/s eta 0:00:05
     ----- ---------------------------------- 2.1/15.3 MB 4.0 MB/s eta 0:00:04
     ------ --------------------------------- 2.5/15.3 MB 


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Инициализация Confluence

In [1]:
from os import environ
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

confluence_token = environ.get('CONFLUENCE_TOKEN')
hf_token = environ.get('HF_TOKEN')
gigachat_token = environ.get('GIGACHAT_TOKEN')

In [2]:
from atlassian import Confluence

confluence_url = "https://confluence.utmn.ru"
confluence = Confluence(url=confluence_url, token=confluence_token)

In [204]:
import pandas as pd

questions = pd.read_csv("questions.csv")[["question", "benchmark"]]
questions

,question,benchmark
0,Где можно оформить справку о подтверждении обу...,Студенты очной формы обучения оформляют справк...
1,Как можно заказать справку об обучении для сту...,Студенты очной формы обучения оформляют справк...
2,Какие документы необходимо иметь при себе для ...,Студенты очной формы обучения оформляют справк...
3,Каким образом можно самостоятельно сформироват...,Сформировать справку-вызов Вы можете самостоят...
4,Каким образом можно узнать о статусе заявки по...,Сформировать справку-вызов Вы можете самостоят...
...,...,...
96,кому отдать договор по физре?,Выбор спортивных секций по Физической культуре...
97,кому отдать договор фитнес-клуба?,Выбор спортивных секций по Физической культуре...
98,как получить справку об обучении?,Студенты очной формы обучения оформляют справк...
99,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...


## Формирование CQL-запроса из вопроса пользователя и поиск online

In [205]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import PyPDFLoader

import spacy
nlp = spacy.load("ru_core_news_sm")
needed_pos = ['NOUN', 'NUM', 'PROPN', 'ADJ', 'VERB', 'X']


def get_cql_query(spaces, question):
    exclude = ' and label != "навигация"' 
    words = [token for token in nlp(question.lower()) if not token.is_stop and
             token.pos_ in needed_pos and len(token.text) > 2]
    if len(words) == 0:
        return ()
    spaces = " or ".join([f"space = {space}" for space in spaces])
    words_with_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                              for word in words])))
    words_without_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                 for word in words if word.pos_ != 'VERB'])))
    words_without_verbs_and_adj = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                         for word in words if word.pos_ not in ['VERB', 'ADJ']])))
    return ("(" + spaces + ") and (" + words_with_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs_and_adj + ")" + exclude)
    
    
def get_document_id(question: str) -> str:
    cql_query = get_cql_query(spaces=["study"], question=question)
    if len(cql_query) == 0:
        return "0"
    results = confluence.cql(cql_query[0], start=0, limit=1)['results']
    if len(results) == 0:
        results = confluence.cql(cql_query[1], start=0, limit=1)['results']
        if len(results) == 0:
            results = confluence.cql(cql_query[2], start=0, limit=1)['results']
            if len(results) == 0:
                return "0"

    return results[0]['content']['id']


def get_document_content_by_id(page_id: str):
    page = confluence.get_page_by_id(page_id, expand='space,body.export_view')
    page_body = page['body']['export_view']['value']
    page_download = page['_links']['base'] + page['_links']['download'] if 'download' in page['_links'].keys() else ''

    try:
        if len(page_body) > 50:
            page_body = page['body']['export_view']['value']
            soup = BeautifulSoup(page_body, 'html.parser')
            page_body_text = soup.get_text(separator=' ')
            content = page_body_text.replace(" \n ", "")
        elif '.pdf' in page_download.lower():
            loader = PyPDFLoader(page_download.split('?')[0])
            content = " ".join([page.page_content for page in loader.load_and_split()])
        else:
            return None
    except:
        return None

    return content


def get_document_content(question: str):
    page_id = get_document_id(question)
    if page_id == "0":
        return None
    return get_document_content_by_id(page_id)

In [206]:
questions["CQL"] = questions["question"].apply(get_document_content)
questions

,question,benchmark,CQL
0,Где можно оформить справку о подтверждении обу...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
1,Как можно заказать справку об обучении для сту...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
2,Какие документы необходимо иметь при себе для ...,Студенты очной формы обучения оформляют справк...,Документ подписан простой электронной подписью...
3,Каким образом можно самостоятельно сформироват...,Сформировать справку-вызов Вы можете самостоят...,Документ подписан простой электронной подписью...
4,Каким образом можно узнать о статусе заявки по...,Сформировать справку-вызов Вы можете самостоят...,None
...,...,...,...
96,кому отдать договор по физре?,Выбор спортивных секций по Физической культуре...,None
97,кому отдать договор фитнес-клуба?,Выбор спортивных секций по Физической культуре...,None
98,как получить справку об обучении?,Студенты очной формы обучения оформляют справк...,"1. Подать заявление о переводе можно лично, об..."
99,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,None


## Создание локального индекса Confluence в PostgreSQL

### Выгрузка и предобработка документов из пространства

In [51]:
pages = confluence.cql("space = study and label != \"навигация\"", start=0, limit=100)['results']
page_ids = [page['content']['id'] for page in pages if 'content' in page.keys()]
conf_df = pd.DataFrame({"page_id": page_ids})
conf_df

,page_id
0,86479083
1,86479057
2,86479066
3,86479079
4,86479078
5,86479077
6,86479076
7,86479075
8,86479074
9,86479073


In [52]:
conf_df["content"] = conf_df["page_id"].apply(get_document_content_by_id)
conf_df

,page_id,content
0,86479083,Сопровождение\nспециализированных\nобразовател...
1,86479057,2 \n \n \n \n1. ОБЩИЕ ПОЛОЖЕНИЯ \n \n1.1. По...
2,86479066,MuHucrepcreo HayKu 14 BbtcuJero o6paaoeaHnR Po...
3,86479079,Об утверждении Регламента \nпроведения промежу...
4,86479078,Об утверждении Р егламента \nпроведения промеж...
5,86479077,MrHucrepcreo HayKn I Bbtctuero o6pasoeaHnR poc...
6,86479076,Об утверждении Регламента \nпроведения промежу...
7,86479075,Об утверждении Регламента \nпроведения промежу...
8,86479074,MrHncrepcreo HayKu 14 Bbtcuero o6paroeaHnn poc...
9,86479073,MrHucrepcreo HayKu t4 Bbtcuero o6pa:oeaHnR Poc...


In [53]:
trash_id = ["86479082", "86479093", "86479069", "86479070", "86479071", "86479072", "86479073", "86479074", "86479077", "86479066"]
conf_df["page_id"] = conf_df["page_id"].apply(lambda x: None if x in trash_id else x)
conf_df = conf_df.dropna().reset_index(drop=True)
conf_df

,page_id,content
0,86479083,Сопровождение\nспециализированных\nобразовател...
1,86479057,2 \n \n \n \n1. ОБЩИЕ ПОЛОЖЕНИЯ \n \n1.1. По...
2,86479079,Об утверждении Регламента \nпроведения промежу...
3,86479078,Об утверждении Р егламента \nпроведения промеж...
4,86479076,Об утверждении Регламента \nпроведения промежу...
5,86479075,Об утверждении Регламента \nпроведения промежу...
6,86479086,Как работать с разделом \n«Выбор модулей»?\nИн...
7,86479085,Документ подписан простой электронной подписью...
8,86479021,Документ подписан простой электронной подписью...
9,86479060,Документ подписан простой электронной подписью...


In [92]:
def lower_stopword_lemmatize(text):
    return " ".join([token.lemma_ for token in nlp(str(text).lower()) if not token.is_stop and token.pos_ != 'PUNCT'])

conf_df["content_lem"] = conf_df.content.apply(lower_stopword_lemmatize)
conf_df

,page_id,content,content_lem
0,86479083,Сопровождение\nспециализированных\nобразовател...,сопровождение \n специализированный \n образов...
1,86479057,2 \n \n \n \n1. ОБЩИЕ ПОЛОЖЕНИЯ \n \n1.1. По...,2 \n \n \n \n 1 общий положение \n \n 1.1 п...
2,86479079,Об утверждении Регламента \nпроведения промежу...,утверждение регламент \n проведение промежуточ...
3,86479078,Об утверждении Р егламента \nпроведения промеж...,утверждение егламента \n проведение промежуточ...
4,86479076,Об утверждении Регламента \nпроведения промежу...,утверждение регламент \n проведение промежуточ...
5,86479075,Об утверждении Регламента \nпроведения промежу...,утверждение регламент \n проведение промежуточ...
6,86479086,Как работать с разделом \n«Выбор модулей»?\nИн...,работать раздел \n выбор модуль \n инструкция ...
7,86479085,Документ подписан простой электронной подписью...,документ подписать простой электронный подпись...
8,86479021,Документ подписан простой электронной подписью...,документ подписать простой электронный подпись...
9,86479060,Документ подписан простой электронной подписью...,документ подписать простой электронный подпись...


### TFIDF

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidfX = tfidf_vectorizer.fit_transform(conf_df.content_lem)
len(tfidf_vectorizer.transform([conf_df.content_lem[0]]).toarray()[0])

2936

### Doc2vec

In [57]:
%pip install gensim --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [95]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([text.split() for text in conf_df.content_lem])]
doc2vec_model = Doc2Vec(documents, vector_size=150, window=3, min_count=1, workers=4)

len(doc2vec_model.infer_vector(conf_df.content_lem[0].split()))

150

### RuBERT-Tiny

In [77]:
import torch
from transformers import AutoTokenizer, AutoModel
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
rubert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
rubert_model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

len(embed_bert_cls(conf_df.content[0], rubert_model, rubert_tokenizer))

312

### RuSBERT-Tiny

In [237]:
from sentence_transformers import SentenceTransformer

rusbert_model = SentenceTransformer('cointegrated/rubert-tiny2')
len(rusbert_model.encode(conf_df.content[0]))

modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
d:\git\virtassist\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timur\.cache\huggingface\hub\models--cointegrated--rubert-tiny2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
README.md: 100%|██████████| 2.19k/2.19k [00:00<?, ?B/s]
sen

312

### Сохранение в БД

In [238]:
from typing import Optional
from pgvector.sqlalchemy import Vector
from sqlalchemy import Text, Column, DateTime, func, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column, relationship

class Base(DeclarativeBase):
    pass

class Document(Base):
    __tablename__ = "document"
    id: Mapped[int] = mapped_column(primary_key=True)
    text: Mapped[str] = mapped_column(Text())
    tfidf: Mapped[Optional[Vector]] = mapped_column(Vector(2936))
    doc2vec: Mapped[Optional[Vector]] = mapped_column(Vector(150))
    rubert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    
    
from sqlalchemy import create_engine
engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=True)
with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    session.commit()
Base.metadata.create_all(engine)
    

2024-02-11 20:51:35,488 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-02-11 20:51:35,489 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 20:51:35,493 INFO sqlalchemy.engine.Engine select current_schema()
2024-02-11 20:51:35,495 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 20:51:35,499 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-02-11 20:51:35,500 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 20:51:35,504 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:51:35,505 INFO sqlalchemy.engine.Engine CREATE EXTENSION IF NOT EXISTS vector
2024-02-11 20:51:35,505 INFO sqlalchemy.engine.Engine [generated in 0.00081s] {}
2024-02-11 20:51:35,509 INFO sqlalchemy.engine.Engine COMMIT
2024-02-11 20:51:35,512 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:51:35,516 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid =

2024-02-11 20:51:35,560 INFO sqlalchemy.engine.Engine COMMIT


In [239]:
with Session(engine) as session:
    for i in range(len(conf_df.content)):
        doc = Document(text=conf_df.content[i], 
                       tfidf=tfidf_vectorizer.transform([conf_df.content_lem[i]]).toarray()[0],
                       doc2vec=doc2vec_model.infer_vector(conf_df.content_lem[i].split()),
                       rubert=embed_bert_cls(conf_df.content[i], rubert_model, rubert_tokenizer),
                       rusbert=rusbert_model.encode(conf_df.content[i]))
        session.add(doc)
    session.commit()   

2024-02-11 20:51:44,356 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:51:44,406 INFO sqlalchemy.engine.Engine INSERT INTO document (text, tfidf, doc2vec, rubert, rusbert) SELECT p0::TEXT, p1::VECTOR(2936), p2::VECTOR(150), p3::VECTOR(312), p4::VECTOR(312) FROM (VALUES (%(text__0)s, %(tfidf__0)s, %(doc2vec__0)s, %(rubert__0)s, %(rusbert__0)s, 0), (%(text__1)s ... 3087 characters truncated ... en(p0, p1, p2, p3, p4, sen_counter) ORDER BY sen_counter RETURNING document.id, document.id AS id__1
2024-02-11 20:51:44,406 INFO sqlalchemy.engine.Engine [generated in 0.04576s (insertmanyvalues) 1/1 (ordered)] {'rusbert__0': '[0.06631461530923843,0.03493586555123329,0.06720124930143356,0.0011828569695353508,-0.020561115816235542,-0.013000190258026123,-0.012056092731654644, ... (6218 characters truncated) ... .02145107090473175,-0.0020276980940252542,0.022892959415912628,0.0029329818207770586,0.04563396796584129,0.0013529256684705615,-0.002223630901426077]', 'tfidf__0': '[0.0,0.0,0.

In [240]:
!docker exec -i db pg_dump -Upostgres virtassist > confluence.dump

### Генерация вопросов к документам через Gigachat

In [123]:
from langchain.prompts import PromptTemplate

prompt_template = """
Задай к тексту в тройных кавычках 3 вопроса.

\"\"\"
{content}
\"\"\"

Вопросы:
"""

prompt = PromptTemplate.from_template(prompt_template)

In [124]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga

In [125]:
new_questions = []
for i in range(13, len(conf_df)):
    query = {"content": conf_df.content[i]}
    questions = giga_chain.invoke(query).strip().split("\n")
    for q in questions:
        new_questions.append({
            "question": q,
            "benchmark": conf_df.content[i]
        })
    print(questions)
new_questions = pd.DataFrame(new_questions)
new_questions

['1. Где можно оформить справку о подтверждении обучения для студентов очной формы?', '2. Как можно заказать справку об обучении для студентов заочной и очно-заочной форм?', '3. Какие документы необходимо иметь при себе для восстановления магнитной карты?']
['1. Каким образом можно самостоятельно сформировать справку-вызов на портале "Вместе"?', '2. Каким образом можно узнать о статусе заявки после ее присвоения?', '3. Какие требования предъявляются к студентам заочной формы обучения для получения справки-вызова на текущую сессию?']
['1. Где находится Единый деканат для подачи заявления на восстановление студенческого билета?', '2. Какие документы необходимо иметь при себе для продления студенческого билета?', '3. Куда обращаться, если магнитная карта (пропуск, проходка) не работает?']
['1. Где можно подать заявку на оформление академической справки?', '2. Каков адрес "Единого деканата"?', '3. Сколько времени занимает процесс подготовки академической справки?']
['1. Где находится Отдел

,question,benchmark
0,1. Где можно оформить справку о подтверждении ...,Студенты очной формы обучения оформляют справк...
1,2. Как можно заказать справку об обучении для ...,Студенты очной формы обучения оформляют справк...
2,3. Какие документы необходимо иметь при себе д...,Студенты очной формы обучения оформляют справк...
3,1. Каким образом можно самостоятельно сформиро...,Сформировать справку-вызов Вы можете самостоят...
4,2. Каким образом можно узнать о статусе заявки...,Сформировать справку-вызов Вы можете самостоят...
...,...,...
70,2. Каким образом можно подать заявление на пре...,Заявление подается через личный кабинет на пор...
71,3. Какие даты указываются в заключении врачебн...,Заявление подается через личный кабинет на пор...
72,1. Какие способы подачи заявления на отчислени...,Подать заявление на отчисление переводом можно...
73,2. Что необходимо приложить к заявлению на отч...,Подать заявление на отчисление переводом можно...


In [200]:
new_questions.question = new_questions.question.apply(lambda x: x[3:])
new_questions

,question,benchmark
0,Где можно оформить справку о подтверждении обу...,Студенты очной формы обучения оформляют справк...
1,Как можно заказать справку об обучении для сту...,Студенты очной формы обучения оформляют справк...
2,Какие документы необходимо иметь при себе для ...,Студенты очной формы обучения оформляют справк...
3,Каким образом можно самостоятельно сформироват...,Сформировать справку-вызов Вы можете самостоят...
4,Каким образом можно узнать о статусе заявки по...,Сформировать справку-вызов Вы можете самостоят...
...,...,...
70,Каким образом можно подать заявление на предос...,Заявление подается через личный кабинет на пор...
71,Какие даты указываются в заключении врачебной ...,Заявление подается через личный кабинет на пор...
72,Какие способы подачи заявления на отчисление п...,Подать заявление на отчисление переводом можно...
73,Что необходимо приложить к заявлению на отчисл...,Подать заявление на отчисление переводом можно...


In [127]:
human_questions = []

In [193]:
question = "как закрыть физкультуру?"
answer = get_document_content(question)
answer

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".\xa0 Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.\xa0 Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.\xa0 Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО!  Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06.09.2023 и будет открыт до конца семестра.\xa0 Для успешной аттестации по дисциплине «Физическая культура:

In [194]:
human_questions.append({
            "question": question,
            "benchmark": answer
        })
human_questions

[{'question': 'где почитать отзывы на элективы?',
  'benchmark': 'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра. Элективы 2023.PDF'},
 {'question': 'когда можно поменять элективы?',
  'benchmark': 'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра. Элективы 2023.PDF'},
 {'question': 'как выбрать электив?',
  'benchmark': 'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как п

In [201]:
questions = pd.concat([new_questions, pd.DataFrame(human_questions)], ignore_index=True)
questions.to_csv("questions.csv")

## Выбор нужного фрагмента через векторный индекс

In [241]:
from sqlalchemy import select

def answer_tfidf(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.tfidf.cosine_distance(
                            tfidf_vectorizer.transform([lower_stopword_lemmatize(question)]).toarray()[0]
                            )).limit(1)).first().text
        
        
def answer_doc2vec(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.doc2vec.cosine_distance(
                           doc2vec_model.infer_vector(lower_stopword_lemmatize(question).split())
                            )).limit(1)).first().text
        
        
def answer_rubert(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.rubert.cosine_distance(
                            embed_bert_cls(question, rubert_model, rubert_tokenizer)
                            )).limit(1)).first().text
        
        
def answer_rusbert(question):
    with Session(engine) as session:
        return session.scalars(select(Document)
                        .order_by(Document.rusbert.cosine_distance(
                            rusbert_model.encode(question)
                            )).limit(1)).first().text

        
print(questions["question"][0])
print(answer_tfidf(questions["question"][0])[:30])
print(answer_doc2vec(questions["question"][0])[:30])
print(answer_rubert(questions["question"][0])[:30])
print(answer_rusbert(questions["question"][0])[:30])

Где можно оформить справку о подтверждении обучения для студентов очной формы?
2024-02-11 20:52:11,505 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:52:11,505 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.tfidf <=> %(tfidf_1)s 
 LIMIT %(param_1)s
2024-02-11 20:52:11,515 INFO sqlalchemy.engine.Engine [generated in 0.00177s] {'tfidf_1': '[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ... (11553 characters truncated) ... ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]', 'param_1': 1}
2024-02-11 20:52:11,523 INFO sqlalchemy.engine.Engine ROLLBACK
Студенты очной формы обучения 
2024-02-11 20:52:11,538 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-1

2024-02-11 20:52:11,571 INFO sqlalchemy.engine.Engine ROLLBACK
Прежде, чем выбрать элективы, 
2024-02-11 20:52:11,585 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:52:11,586 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.rusbert <=> %(rusbert_1)s 
 LIMIT %(param_1)s
2024-02-11 20:52:11,587 INFO sqlalchemy.engine.Engine [generated in 0.00118s] {'rusbert_1': '[0.05451228842139244,-0.07075323164463043,-0.037235673516988754,-0.0591459721326828,-0.0346173532307148,-0.023661581799387932,-0.06194758042693138,0. ... (6242 characters truncated) ... -0.059011902660131454,0.025828158482909203,0.019221536815166473,0.09308307617902756,-0.01870676502585411,-0.009041265584528446,-0.013084802776575089]', 'param_1': 1}
2024-02-11 20:52:11,595 INFO sqlalchemy.engine.Engine ROLLBACK
Сформировать справку-вызов Вы 


In [207]:
questions["tfidf"] = questions["question"].apply(answer_tfidf)
questions

2024-02-11 20:21:18,560 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:21:18,562 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.tfidf <=> %(tfidf_1)s 
 LIMIT %(param_1)s
2024-02-11 20:21:18,563 INFO sqlalchemy.engine.Engine [cached since 5585s ago] {'tfidf_1': '[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ... (11553 characters truncated) ... ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]', 'param_1': 1}
2024-02-11 20:21:18,570 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-11 20:21:18,585 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:21:18,587 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2

2024-02-11 20:21:18,594 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-11 20:21:18,615 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:21:18,619 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.tfidf <=> %(tfidf_1)s 
 LIMIT %(param_1)s
2024-02-11 20:21:18,620 INFO sqlalchemy.engine.Engine [cached since 5585s ago] {'tfidf_1': '[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0, ... (11507 characters truncated) ... ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]', 'param_1': 1}
2024-02-11 20:21:18,669 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-11 20:21:18,689 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:21:18,691 INFO sqlalchemy.engine.Engine S

,question,benchmark,CQL,tfidf
0,Где можно оформить справку о подтверждении обу...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
1,Как можно заказать справку об обучении для сту...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
2,Какие документы необходимо иметь при себе для ...,Студенты очной формы обучения оформляют справк...,Документ подписан простой электронной подписью...,Для восстановления студенческого билета Вам ...
3,Каким образом можно самостоятельно сформироват...,Сформировать справку-вызов Вы можете самостоят...,Документ подписан простой электронной подписью...,Сформировать справку-вызов Вы можете самостоят...
4,Каким образом можно узнать о статусе заявки по...,Сформировать справку-вызов Вы можете самостоят...,None,Сформировать справку-вызов Вы можете самостоят...
...,...,...,...,...
96,кому отдать договор по физре?,Выбор спортивных секций по Физической культуре...,None,"При восстановлении на договорное место, после ..."
97,кому отдать договор фитнес-клуба?,Выбор спортивных секций по Физической культуре...,None,"При восстановлении на договорное место, после ..."
98,как получить справку об обучении?,Студенты очной формы обучения оформляют справк...,"1. Подать заявление о переводе можно лично, об...",Для оформления академической справки (справки ...
99,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,None,Для оформления академической справки (справки ...


In [208]:
questions["doc2vec"] = questions["question"].apply(answer_doc2vec)
questions

2024-02-11 20:21:40,655 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:21:40,657 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.doc2vec <=> %(doc2vec_1)s 
 LIMIT %(param_1)s
2024-02-11 20:21:40,658 INFO sqlalchemy.engine.Engine [cached since 5199s ago] {'doc2vec_1': '[-0.0057290405966341496,-0.018747182562947273,0.03036096692085266,-0.004733955953270197,-0.0009093501721508801,0.014438980259001255,-0.02286250330507 ... (2913 characters truncated) ... .01753091998398304,-0.009621789678931236,0.02760932594537735,-0.006624775473028421,-0.0008653843542560935,0.0017356877215206623,-0.03501250222325325]', 'param_1': 1}
2024-02-11 20:21:40,664 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-11 20:21:40,680 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:21:40,681 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document

,question,benchmark,CQL,tfidf,doc2vec
0,Где можно оформить справку о подтверждении обу...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,"По вопросам оплаты обучения, суммы задолженнос..."
1,Как можно заказать справку об обучении для сту...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,"По вопросам оплаты обучения, суммы задолженнос..."
2,Какие документы необходимо иметь при себе для ...,Студенты очной формы обучения оформляют справк...,Документ подписан простой электронной подписью...,Для восстановления студенческого билета Вам ...,"При восстановлении на договорное место, после ..."
3,Каким образом можно самостоятельно сформироват...,Сформировать справку-вызов Вы можете самостоят...,Документ подписан простой электронной подписью...,Сформировать справку-вызов Вы можете самостоят...,"Прежде, чем выбрать элективы, рекомендуем почи..."
4,Каким образом можно узнать о статусе заявки по...,Сформировать справку-вызов Вы можете самостоят...,None,Сформировать справку-вызов Вы можете самостоят...,"По вопросу получения справки о доходах, размер..."
...,...,...,...,...,...
96,кому отдать договор по физре?,Выбор спортивных секций по Физической культуре...,None,"При восстановлении на договорное место, после ...","Восстановление лица , ранее обучавшегося за сч..."
97,кому отдать договор фитнес-клуба?,Выбор спортивных секций по Физической культуре...,None,"При восстановлении на договорное место, после ...",Пакет документов для подачи заявления на перев...
98,как получить справку об обучении?,Студенты очной формы обучения оформляют справк...,"1. Подать заявление о переводе можно лично, об...",Для оформления академической справки (справки ...,"1. Подать заявление о переводе можно лично, об..."
99,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,None,Для оформления академической справки (справки ...,Документ подписан простой электронной подписью...


In [209]:
questions["rubert"] = questions["question"].apply(answer_rubert)
questions

2024-02-11 20:21:49,872 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:21:49,874 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.rubert <=> %(rubert_1)s 
 LIMIT %(param_1)s
2024-02-11 20:21:49,874 INFO sqlalchemy.engine.Engine [cached since 5208s ago] {'rubert_1': '[0.10502980649471283,-0.06491471081972122,-0.03736669570207596,-0.053122378885746,-0.004864814225584269,-0.052283331751823425,-0.020825831219553947,0 ... (6240 characters truncated) ... 0.03618717938661575,0.022618453949689865,0.007147232536226511,0.057576533406972885,-0.018419669941067696,-0.022143252193927765,0.0035743231419473886]', 'param_1': 1}
2024-02-11 20:21:49,881 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-11 20:21:49,892 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:21:49,894 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.do

,question,benchmark,CQL,tfidf,doc2vec,rubert
0,Где можно оформить справку о подтверждении обу...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,"По вопросам оплаты обучения, суммы задолженнос...","Прежде, чем выбрать элективы, рекомендуем почи..."
1,Как можно заказать справку об обучении для сту...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,"По вопросам оплаты обучения, суммы задолженнос...","Прежде, чем выбрать элективы, рекомендуем почи..."
2,Какие документы необходимо иметь при себе для ...,Студенты очной формы обучения оформляют справк...,Документ подписан простой электронной подписью...,Для восстановления студенческого билета Вам ...,"При восстановлении на договорное место, после ...","При восстановлении на договорное место, после ..."
3,Каким образом можно самостоятельно сформироват...,Сформировать справку-вызов Вы можете самостоят...,Документ подписан простой электронной подписью...,Сформировать справку-вызов Вы можете самостоят...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
4,Каким образом можно узнать о статусе заявки по...,Сформировать справку-вызов Вы можете самостоят...,None,Сформировать справку-вызов Вы можете самостоят...,"По вопросу получения справки о доходах, размер...","При восстановлении на договорное место, после ..."
...,...,...,...,...,...,...
96,кому отдать договор по физре?,Выбор спортивных секций по Физической культуре...,None,"При восстановлении на договорное место, после ...","Восстановление лица , ранее обучавшегося за сч...","При восстановлении на договорное место, после ..."
97,кому отдать договор фитнес-клуба?,Выбор спортивных секций по Физической культуре...,None,"При восстановлении на договорное место, после ...",Пакет документов для подачи заявления на перев...,Для рассмотрения возможности продления академи...
98,как получить справку об обучении?,Студенты очной формы обучения оформляют справк...,"1. Подать заявление о переводе можно лично, об...",Для оформления академической справки (справки ...,"1. Подать заявление о переводе можно лично, об...","Прежде, чем выбрать элективы, рекомендуем почи..."
99,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,None,Для оформления академической справки (справки ...,Документ подписан простой электронной подписью...,Для рассмотрения возможности продления академи...


In [243]:
questions["rusbert"] = questions["question"].apply(answer_rusbert)
questions

2024-02-11 20:53:35,292 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:53:35,292 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.rusbert <=> %(rusbert_1)s 
 LIMIT %(param_1)s
2024-02-11 20:53:35,294 INFO sqlalchemy.engine.Engine [cached since 83.71s ago] {'rusbert_1': '[0.05451228842139244,-0.07075323164463043,-0.037235673516988754,-0.0591459721326828,-0.0346173532307148,-0.023661581799387932,-0.06194758042693138,0. ... (6242 characters truncated) ... -0.059011902660131454,0.025828158482909203,0.019221536815166473,0.09308307617902756,-0.01870676502585411,-0.009041265584528446,-0.013084802776575089]', 'param_1': 1}
2024-02-11 20:53:35,302 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-11 20:53:35,315 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:53:35,316 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, documen

2024-02-11 20:53:35,345 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-11 20:53:35,353 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:53:35,362 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.rusbert <=> %(rusbert_1)s 
 LIMIT %(param_1)s
2024-02-11 20:53:35,363 INFO sqlalchemy.engine.Engine [cached since 83.78s ago] {'rusbert_1': '[-0.006520435214042664,-0.001916915294714272,-0.005785895045846701,-0.09045987576246262,-0.03748404234647751,0.035370782017707825,-0.0437302812933921 ... (6255 characters truncated) ... 3,-0.022734297439455986,0.012979026883840561,0.0421079620718956,0.05458439141511917,0.0019743710290640593,0.048128318041563034,-0.013222468085587025]', 'param_1': 1}
2024-02-11 20:53:35,370 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-11 20:53:35,383 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 20:53:35,383 INFO sqlalchemy.engine.En

,question,benchmark,CQL,tfidf,doc2vec,rubert,rusbert
0,Где можно оформить справку о подтверждении обу...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,"По вопросам оплаты обучения, суммы задолженнос...","Прежде, чем выбрать элективы, рекомендуем почи...",Сформировать справку-вызов Вы можете самостоят...
1,Как можно заказать справку об обучении для сту...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...,"По вопросам оплаты обучения, суммы задолженнос...","Прежде, чем выбрать элективы, рекомендуем почи...",Подать заявление на отчисление переводом можно...
2,Какие документы необходимо иметь при себе для ...,Студенты очной формы обучения оформляют справк...,Документ подписан простой электронной подписью...,Для восстановления студенческого билета Вам ...,"При восстановлении на договорное место, после ...","При восстановлении на договорное место, после ...",Для восстановления студенческого билета Вам ...
3,Каким образом можно самостоятельно сформироват...,Сформировать справку-вызов Вы можете самостоят...,Документ подписан простой электронной подписью...,Сформировать справку-вызов Вы можете самостоят...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Сформировать справку-вызов Вы можете самостоят...
4,Каким образом можно узнать о статусе заявки по...,Сформировать справку-вызов Вы можете самостоят...,None,Сформировать справку-вызов Вы можете самостоят...,"По вопросу получения справки о доходах, размер...","При восстановлении на договорное место, после ...","При восстановлении на договорное место, после ..."
...,...,...,...,...,...,...,...
96,кому отдать договор по физре?,Выбор спортивных секций по Физической культуре...,None,"При восстановлении на договорное место, после ...","Восстановление лица , ранее обучавшегося за сч...","При восстановлении на договорное место, после ...",Пакет документов для подачи заявления на перев...
97,кому отдать договор фитнес-клуба?,Выбор спортивных секций по Физической культуре...,None,"При восстановлении на договорное место, после ...",Пакет документов для подачи заявления на перев...,Для рассмотрения возможности продления академи...,Заявление подается через личный кабинет на пор...
98,как получить справку об обучении?,Студенты очной формы обучения оформляют справк...,"1. Подать заявление о переводе можно лично, об...",Для оформления академической справки (справки ...,"1. Подать заявление о переводе можно лично, об...","Прежде, чем выбрать элективы, рекомендуем почи...",Сформировать справку-вызов Вы можете самостоят...
99,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,None,Для оформления академической справки (справки ...,Документ подписан простой электронной подписью...,Для рассмотрения возможности продления академи...,Подать заявление на отчисление переводом можно...


In [244]:
questions.to_csv("questions_documents.csv")

### Выбор лучшего алгоритма: BERTScore

In [213]:
%pip install bert_score --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [214]:
from bert_score import BERTScorer

scorer = BERTScorer(lang="ru")

tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 53.7kB/s]
d:\git\virtassist\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timur\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config.json: 100%|██████████| 625/625 [

In [225]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.benchmark))
f"benchmark. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'benchmark. Precision: 1.000 Recall: 1.000 F1 score: 1.000'

In [226]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.CQL.apply(lambda x: "" if x is None else x)))
f"CQL. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'CQL. Precision: 0.680 Recall: 0.657 F1 score: 0.667'

In [232]:
P, R, F1 = scorer.score(list(questions.benchmark[75:]), list(questions.CQL.apply(lambda x: "" if x is None else x))[75:])
f"CQL. HUMAN. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'CQL. HUMAN. Precision: 0.724 Recall: 0.720 F1 score: 0.722'

In [227]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.tfidf))
f"TFIDF. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'TFIDF. Precision: 0.889 Recall: 0.891 F1 score: 0.889'

In [233]:
P, R, F1 = scorer.score(list(questions.benchmark[75:]), list(questions.tfidf[75:]))
f"TFIDF. HUMAN. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'TFIDF. HUMAN. Precision: 0.829 Recall: 0.853 F1 score: 0.840'

In [228]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.doc2vec))
f"doc2vec. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'doc2vec. Precision: 0.681 Recall: 0.686 F1 score: 0.682'

In [234]:
P, R, F1 = scorer.score(list(questions.benchmark[75:]), list(questions.doc2vec[75:]))
f"doc2vec. HUMAN. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'doc2vec. HUMAN. Precision: 0.681 Recall: 0.687 F1 score: 0.683'

In [229]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.rubert))
f"rubert. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'rubert. Precision: 0.726 Recall: 0.756 F1 score: 0.740'

In [235]:
P, R, F1 = scorer.score(list(questions.benchmark[75:]), list(questions.rubert[75:]))
f"rubert. HUMAN. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'rubert. HUMAN. Precision: 0.701 Recall: 0.719 F1 score: 0.709'

In [245]:
P, R, F1 = scorer.score(list(questions.benchmark), list(questions.rusbert))
f"rusbert. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'rusbert. Precision: 0.799 Recall: 0.809 F1 score: 0.803'

In [246]:
P, R, F1 = scorer.score(list(questions.benchmark[75:]), list(questions.rusbert[75:]))
f"rusbert. HUMAN. Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'rusbert. HUMAN. Precision: 0.720 Recall: 0.736 F1 score: 0.727'

In [231]:
questions.question.to_csv("oq.csv")

## Составление промпта

In [247]:
from langchain.prompts import PromptTemplate

prompt_template = """
Используй следующий текст в тройных кавычках, чтобы кратко ответить на вопрос студента в конце. 
Не изменяй и не убирай ссылки, адреса и телефоны. Если ты не можешь найти ответ, напиши, что ответ не найден.
Ответ не должен превышать 100 слов.

\"\"\"
{context}
\"\"\"

Вопрос: {question}
"""

prompt = PromptTemplate.from_template(prompt_template)

## GigaChat

In [248]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga

In [271]:
# question = questions.question[89]
question = "Самый крутой сервис по выбору авиабилетов?"
print(question, end="\n\n")
document = answer_rusbert(question)
print(document)

Самый крутой сервис по выбору авиабилетов?

2024-02-11 21:06:44,384 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 21:06:44,386 INFO sqlalchemy.engine.Engine SELECT document.id, document.text, document.tfidf, document.doc2vec, document.rubert, document.rusbert 
FROM document ORDER BY document.rusbert <=> %(rusbert_1)s 
 LIMIT %(param_1)s
2024-02-11 21:06:44,386 INFO sqlalchemy.engine.Engine [cached since 872.8s ago] {'rusbert_1': '[0.005533029790967703,0.013334708288311958,-0.03015470691025257,-0.1701069474220276,-0.043242618441581726,0.04188287630677223,-0.031845852732658386,0 ... (6246 characters truncated) ... 24,-0.011155599728226662,0.009449596516788006,-0.02684507891535759,0.05969000235199928,0.012244931422173977,0.05982932820916176,-0.09153693169355392]', 'param_1': 1}
2024-02-11 21:06:44,393 INFO sqlalchemy.engine.Engine ROLLBACK
Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём 

In [272]:
query = {"context": document,
        "question": question}
print(giga_chain.invoke(query).strip())
print()
print(query)

Ответ не найден.

{'context': 'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе «Отзывус»:  https://electives.utmn.ru . Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра. Элективы 2023.PDF', 'question': 'Самый крутой сервис по выбору авиабилетов?'}


## Большие языковые генеративные модели

In [25]:
import torch
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

device = torch.cuda.current_device() if torch.cuda.is_available() and torch.cuda.mem_get_info()[0] >= 2*1024**3 else -1
device

d:\git\virtassist\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0

In [26]:
# FOR SAVING MODEL ONLY!
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# model_name = "IlyaGusev/fred_t5_ru_turbo_alpaca"
# model_name = "ai-forever/FRED-T5-large"
# model_name = "ai-forever/FRED-T5-1.7B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# pipe = pipeline(
#     "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500
# )
# pipe.save_pretrained("saved_models/FRED-T5-1.7B")

tokenizer_config.json: 100%|██████████| 20.2k/20.2k [00:00<00:00, 15.1MB/s]
d:\git\virtassist\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Timur\.cache\huggingface\hub\models--ai-forever--FRED-T5-1.7B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.json: 100%|██████████| 1.71M/1.71M 

In [27]:
saved_pipeline = pipeline("text2text-generation", "saved_models/FRED-T5-1.7B", device=device, max_new_tokens=10000)
hf_model = HuggingFacePipeline(pipeline=saved_pipeline).bind(stop=["\n\n"])
gpu_chain = prompt | hf_model

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.51s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
content = found_doc['page_content']
query = {"context": content,
        "question": question}
print(gpu_chain.invoke(query).strip())
print()
print(query)

Вопрос: Как мне узнать, что я записан на модуль?
1.Перейти в раздел ИОТ –Выбор модулей
2.В случае, если окно выбора открыто только на просмотр, 
дождаться даты и времени открытия окна на запись
3.Выбрать интересующую дисциплину
4.Осуществить запись в каждый из типов учебных команд, 
доступных для выбора (например, лекционные команды или команды 
по практике)
5.Нажать кнопку Выбрать
6.В случае успешной записи на дисциплину в правом верхнем 
углу появляется всплывающее окно об оформленном выборе
7.Рядом с выбранной дисциплиной появляется значок
 |Критерий завершения уже выполнен, выбрано необходимое количество 
дисциплин или на необходимое количество единиц ценности
8.Рядом с выбранной дисциплиной появляется значок
 |Недостаточно свободных мест в учебной команде
9.Во всплывающем окне выбрать вариант Да, отписаться
10.Отмена записи на дисциплину
11.Просмотр описания модуля
12.Просмотр расписания модуля и всех выбранных модулей
13.Просмотр состава команды
14.Отмена записи на дисциплину
15.

### Гигиена

In [18]:
del gpu_chain
del hf_model
del saved_pipeline
torch.cuda.empty_cache()